In [65]:
import torch
import pandas as pd
from gridFM.datasets.powergrid import GridDatasetMem
from gridFM.datasets.data_normalization import *
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_torch_coo_tensor
from torch.utils.data import ConcatDataset
import torch.nn.functional as F
from gridFM.models.graphTransformer import GraphTransformer
from tqdm import tqdm
from torch.utils.data import Subset
from gridFM.utils.loss import *
from gridFM.io.param_handler import *
import random
from gridFM.datasets.utils import split_dataset

torch.set_printoptions(precision=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [66]:
torch.manual_seed(200)
random.seed(200)
np.random.seed(200)

In [67]:
data_path1 = r"C:\Users\MatteoMazzonelli\Desktop\gridfm_model_evaluation\scripts\..\data\case300"
scenarios = 500
norm_method = "baseMVAnorm"
node_normalizer, edge_normalizer = BaseMVANormalizer(True), BaseMVANormalizer(False) 
mask_ratio = 0.5
mask_dim = 6
mask_value = 0

In [68]:
dataset = GridDatasetMem(
        root=data_path1,
        norm_method=norm_method,
        node_normalizer=node_normalizer,
        edge_normalizer=edge_normalizer,
        mask_ratio=mask_ratio,
        mask_dim=mask_dim,
    )

In [69]:
train_dataset, val_dataset, test_dataset = split_dataset(
                dataset, data_path1, 0.1, 0.1
            )

In [70]:
loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [71]:
loss = PBELoss()

In [72]:
loss_sum = 0.0
power_loss_sum = 0.0
active_power_loss_sum = 0.0
reactive_power_loss_sum = 0.0
num_batches = 0

# Loop through the batches
for batch in tqdm(loader):
    metric = loss(batch.y, batch.y, batch.edge_index, batch.edge_attr, batch.mask)
    
    # Accumulate the metrics
    loss_sum += metric['loss'].item()
    power_loss_sum += metric['Power power loss in p.u.']
    active_power_loss_sum += metric['Active Power Loss in p.u.']
    reactive_power_loss_sum += metric['Reactive Power Loss in p.u.']
    num_batches += 1

# Calculate the means
mean_loss = loss_sum / num_batches
mean_power_loss = power_loss_sum / num_batches
mean_active_power_loss = active_power_loss_sum / num_batches
mean_reactive_power_loss = reactive_power_loss_sum / num_batches

# Print the results

    

100%|██████████| 118/118 [01:19<00:00,  1.49it/s]


In [73]:
print("BaseMVA: ", node_normalizer.baseMVA)

BaseMVA:  tensor(2465.)


In [74]:
print(f"Mean Power Power Loss in p.u.: {mean_power_loss*node_normalizer.baseMVA}")
print(f"Mean Active Power Loss in p.u.: {mean_active_power_loss*node_normalizer.baseMVA}")
print(f"Mean Reactive Power Loss in p.u.: {mean_reactive_power_loss*node_normalizer.baseMVA}")

Mean Power Power Loss in p.u.: 0.001269189640879631
Mean Active Power Loss in p.u.: 0.0003050217928830534
Mean Reactive Power Loss in p.u.: 0.0011681437026709318
